TODO
- genomic data needs to be PCA and split up into train/val/data
- get rid of acc

In [3]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

# can revome the lines that need these
import os
import pickle

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import utils
from torch.utils.tensorboard import SummaryWriter

from datasets import GeneralDataset
from Transforms import get_transformations
from utils import get_data_splits, get_input_params
from train_mtl import mtl_experiment

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [4]:
seg_4class_weights = np.load('../data/segmentation_4-class_weights.npy')
seg_4class_weights = torch.FloatTensor(seg_4class_weights).to(device)

NameError: name 'device' is not defined

In [5]:
seg_4class_weights

array([ 1.        ,  8.9263424 ,  7.79622053, 31.17438108])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Choose parameters
- 4 channel full resolution:
```python
    task \in ['idh', '1p19q']
    dataformat \in ['raw3D', 'crop3Dslice', 'modality3D']
    modality \in ['t1ce', 'flair', 't2', 't1', 't1ce-t1']
    include_genomic_data \in [True, False]
```

In [ ]:
task = 'idh'
dataformat = 'crop3Dslice'
modality = 'All' # only relevent for 'modality3D' dataformat
include_genomic_data = False # don't include genomic data

null_genomic = not include_genomic_data

dataformat, channels, resize_shape = get_input_params(dataformat)

In [ ]:
if channels == 4:
    seg_classes = 4
else:
    seg_classes = 2

In [ ]:
print('task:\t\t', task)
print('dataformat:\t', dataformat)
print('channels:\t', channels)
print('modality:\t', modality)
print('resize_shape:\t', resize_shape)
print('null_genomic:\t', null_genomic)

In [1]:
# MRI directory
image_dir = '../data/all_brats_scans/'

# metadata for all brats (including tcia) data
best_model_loc = '../pretrained/espnet_3d_brats.pth' # segmentation model weights
glioma_metadata_df = pd.read_csv('../data/glioma_metadata.csv', index_col=0) # metadata file

# get training splits
train_df, val_df, classes = get_data_splits(metadata_df=glioma_metadata_df, task=task, mtl=False)

# map between brats dataset and tcia data (tcia data is avalible for a subset of the brats patients)
brats2tcia_df = glioma_metadata_df['tciaID']
# brats2tcia_df = pd.read_csv('../../miccai_clean/data/brats2tcia_df_542x1.csv', index_col=0)

# these are labeled files (they were paths in old dataloader) but they are dataframes
labels_dict = {'train':train_df, 'val':val_df, 'data':glioma_metadata_df}

genomic_data_dict = {'train':'../data/MGL/MGL_235x50.csv', 'val':'../data/MGL/MGL_235x50.csv'}

label = task

print('Train size', len(train_df))

NameError: name 'pd' is not defined

# Dataloader

In [7]:
# batch size
train_transformations, seg_transformations, val_transformations = get_transformations(channels=channels, 
                                                                                      resize_shape=resize_shape, 
                                                                                      prob_voxel_zero=0.2, 
                                                                                      prob_true=0.8, 
                                                                                      prob_channel_zero=0.5,
                                                                                      mtl=False)
data_transforms = {'train': train_transformations, 'val':   val_transformations}
                   
transformed_dataset_train = GeneralDataset(metadata_df=train_df, 
                                           root_dir=image_dir,
                                           genomic_csv_file = genomic_data_dict['train'],
                                           transform=train_transformations,
                                           seg_transform=seg_transformations,
                                           label=label,
                                           classes=classes,
                                           dataformat=dataformat,
                                           returndims=resize_shape,
                                           brats2tcia_df=brats2tcia_df,
                                           null_genomic = null_genomic,
                                           pretrained=best_model_loc,
                                           modality=modality)

transformed_dataset_val = GeneralDataset(metadata_df=val_df,
                                         root_dir=image_dir,
                                         genomic_csv_file = genomic_data_dict['val'],
                                         transform=val_transformations,
                                         seg_transform=seg_transformations,
                                         label=label,
                                         classes=classes,
                                         dataformat=dataformat,
                                         returndims=resize_shape,
                                         brats2tcia_df=brats2tcia_df,
                                         null_genomic = null_genomic,
                                         pretrained=best_model_loc,
                                         modality=modality)


image_datasets = {'train':transformed_dataset_train, 'val':transformed_dataset_val}

train_batch_size, val_batch_size = 4, 4
dataloader_train = DataLoader(image_datasets['train'], batch_size=train_batch_size, shuffle=True, num_workers=4)
dataloader_val = DataLoader(image_datasets['val'], batch_size=val_batch_size, shuffle=True, num_workers=4)

dataloaders = {'train':dataloader_train, 'val':dataloader_val}
dataset_sizes = {'train':len(image_datasets['train']), 'val':len(image_datasets['val'])}

class_names = image_datasets['train'].classes
class_names

['wildtype', 'mutant']

# Visualize Data

In [8]:
# # visualize training (or validation) data
# for i, data in enumerate(dataloaders['train']):
#     # data batch
#     (image, seg_image, genomic_data, seg_probs), label, (OS, OS_EVENT), bratsID = data
#     # print scan ID
#     print(bratsID[0])
    
#     # format MRI images (slices of volumetric input)
#     img = image[0,:, :, :, int(image.shape[-1]/2)].squeeze()
#     img = utils.make_grid(img)
#     img = img.detach().cpu().numpy()
    
#     # plot images
#     plt.figure(figsize=(15, 8))
#     img_list = [img[i].T for i in range(channels)] # 1 image per channel
#     plt.imshow(np.hstack(img_list), cmap='Greys_r')
#     plt.show()

#     ## plot segmentation mask ##
#     seg_img = seg_image[0, :, :, :, int(seg_image.shape[-1]/2)].squeeze()
#     seg_img = utils.make_grid(seg_img).detach().cpu().numpy()

#     plt.figure(figsize=(4, 4))
#     plt.imshow(np.hstack([seg_img[0].T]), cmap='Greys_r')
#     plt.show()

#     break

In [11]:
# tensorboard
img_dims = str(resize_shape[0]) + 'x' + str(resize_shape[1]) + 'x' + str(resize_shape[2])
weight_outfile_prefix = '3D_' + task + '_' + modality + '_' \
                    + img_dims + '_genomic-' + str(include_genomic_data)
print('tensorboad:', weight_outfile_prefix)
writer = SummaryWriter('runs1/'+weight_outfile_prefix)


# get loss weights
num_classes = 2
label_df = train_df.loc[train_df[task].isin([0,1])][task]
_, cnts = np.unique(label_df, return_counts=True)
loss_weights = (np.ones(num_classes)/cnts)*np.max(cnts)
loss_weights = torch.FloatTensor(loss_weights).to(device)
# criterion = nn.CrossEntropyLoss(weight=loss_weights)
print('loss weights:', loss_weights)


# train
best_auc_list, best_acc_list = [], []
epochs = 50
iterations = 10
seg_loss_weight = 0.1
surv_loss_weight = None


brats_seg_ids = glioma_metadata_df[glioma_metadata_df['gt_seg'] == 1].index

seg_4class_weights = np.load('../data/segmentation_4-class_weights.npy')
seg_4class_weights = torch.FloatTensor(seg_4class_weights).to(device)
#     print('seg_4class_weights:', seg_4class_weights)

seg_2class_weights = np.load('../data/segmentation_2-class_weights.npy')
seg_2class_weights = torch.FloatTensor(seg_2class_weights).to(device)
#     print('seg_2class_weights:', seg_2class_weights)


for i in range(iterations):
    print('Iteration', i)

    best_auc_list, best_acc_list = mtl_experiment(dataloaders=dataloaders,
                                                  data_transforms=data_transforms,
                                                  dataset_sizes=dataset_sizes,
                                                  best_model_loc=best_model_loc, 
                                                  best_auc_list=best_auc_list,
                                                  best_acc_list=best_acc_list,
                                                  weight_outfile_prefix=weight_outfile_prefix,
                                                  class_names=class_names,
                                                  channels=channels, 
                                                  loss_weights=loss_weights,
                                                  seg_4class_weights=seg_4class_weights,
                                                  seg_2class_weights=seg_2class_weights,
                                                  seg_loss_weight=seg_loss_weight,
                                                  surv_loss_weight=surv_loss_weight,
                                                  device=device,
                                                  brats_seg_ids=brats_seg_ids,
                                                  writer=writer,
                                                  model_weights_dir='../model_weights/results/',
                                                  epochs=epochs,
                                                  iterations=iterations,
                                                  standard_unlabled_loss=True,
                                                  include_genomic_data=include_genomic_data,
                                                  modality=modality,
                                                  take_surv_loss=False,
                                                  g_in_features=50,
                                                  g_out_features=128)


tensorboad: 3D_idh_All_64x64x64_genomic-False
loss weights: tensor([1.0364, 1.0000], device='cuda:0')
Iteration 0


New Best AUC-acc average:	 0.6765046296296295 	in epoch 0
New Best Dice:	 0.5596956121317919 	in epoch 0
New Best ACC:	 0.6516203703703703 	in epoch 0
New Best AUC-acc average:	 0.8081597222222223 	in epoch 1
New Best ACC:	 0.7667824074074074 	in epoch 1
New Best AUC-acc average:	 0.837962962962963 	in epoch 5
New Best ACC:	 0.8321759259259259 	in epoch 5
New Best AUC-acc average:	 0.8388310185185185 	in epoch 6
New Best AUC:	 0.8483796296296295 	in epoch 6
New Best AUC:	 0.8530092592592593 	in epoch 15
Epoch    17: reducing learning rate of group 0 to 5.0000e-05.
New Best AUC:	 0.857638888888889 	in epoch 24
New Best AUC:	 0.8634259259259259 	in epoch 26
New Best AUC:	 0.8726851851851852 	in epoch 28
Epoch    37: reducing learning rate of group 0 to 5.0000e-06.
New Best AUC-acc average:	 0.8449074074074074 	in epoch 39
New Best AUC:	 0.883101851851852 	in epoch 41
Epoch    48: reducing learning rate of group 0 to 5.0000e-07.

Finished Training
Iteration 1


New Best AUC-acc average:	 0.6024305555555556 	in epoch 0
New Best Dice:	 0.566708255899686 	in epoch 0
New Best ACC:	 0.5 	in epoch 0
New Best AUC-acc average:	 0.6354166666666667 	in epoch 1
New Best AUC-acc average:	 0.6412037037037037 	in epoch 2
New Best AUC-acc average:	 0.6629050925925926 	in epoch 3
New Best ACC:	 0.572337962962963 	in epoch 3
New Best ACC:	 0.6469907407407407 	in epoch 4
New Best AUC-acc average:	 0.6820023148148148 	in epoch 5
New Best AUC:	 0.6782407407407408 	in epoch 6
New Best AUC-acc average:	 0.7980324074074074 	in epoch 7
New Best AUC:	 0.8194444444444444 	in epoch 7
New Best ACC:	 0.7766203703703703 	in epoch 7
New Best AUC:	 0.8344907407407408 	in epoch 8
New Best AUC:	 0.8622685185185185 	in epoch 9
Epoch    24: reducing learning rate of group 0 to 5.0000e-05.
Epoch    35: reducing learning rate of group 0 to 5.0000e-06.



KeyboardInterrupt: 

In [1]:
!nvidia-smi

Fri Jan  8 17:01:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 39%   46C    P8    21W / 250W |   3776MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------